In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd

In [2]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
data1=pd.DataFrame(data)
target = raw_df.values[1::2, 2]
# data1["PRICE"]=target
data1.rename(columns={0:"CRIM",1:"ZN",2:"INDUS",3:"CHAS",4:"NOX",5:"RM",6:"AGE",7:"DIS",8:"RAD",9:"TAX",10:"PTRATIO",11:"B",12:"LSTAT"},inplace=True)
features=data1.drop(["INDUS","AGE"],axis=1)
features

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,0.0,0.573,6.593,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,0.0,0.573,6.120,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,0.0,0.573,6.976,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,0.0,0.573,6.794,2.3889,1.0,273.0,21.0,393.45,6.48


In [3]:
target_log=pd.DataFrame(data=np.log(target),columns=["PRICE"])
target_log

,PRICE
0,3.178054
1,3.072693
2,3.546740
3,3.508556
4,3.589059
...,...
501,3.109061
502,3.025291
503,3.173878
504,3.091042


In [4]:
property_stats=features.mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [5]:
reg=LinearRegression().fit(features,target_log)
fitted_vals=reg.predict(features)
MSE=mean_squared_error(target_log,fitted_vals)
RMSE=np.sqrt(MSE)

In [19]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8
def log_price_estimate(nr_roms,students_per_classroom,next_to_river=False,high_confidence=True):
    property_stats[0][RM_IDX]=nr_roms
    property_stats[0][PTRATIO_IDX]=students_per_classroom
    if next_to_river:
        property_stats[0][CHAS_IDX]=1
    else:
        property_stats[0][CHAS_IDX]=0
    log_estimate=reg.predict(property_stats)[0][0]
    if high_confidence:
        upper_bound=log_estimate+2*RMSE
        lower_bound=log_estimate-2*RMSE
        interval=95
    else:
        upper_bound=log_estimate+RMSE
        lower_bound=log_estimate-RMSE
        interval=68
    return log_estimate,upper_bound,lower_bound,interval
log_price_estimate(3,20,next_to_river=True,high_confidence=True)
# 3.03451287

c:\Users\Shubham\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.7767581914803996, 3.15178246187466, 2.401733921086139, 95)

In [11]:
np.median(target)

21.2

In [8]:
ZILLOW_MEDIAN_PRICE=583.3
SCALER_FACTOR=ZILLOW_MEDIAN_PRICE/np.median(target)
def get_dollar_estimate(rm,ptratio,chas=False,large_range=True):
    """Estimate the price of a property in boston"""
    if rm<1 or ptratio<1:
        print("This is unrealistic. Please try again.")
        return
    log_estimate,upper,lower,conf=log_price_estimate(rm,ptratio,next_to_river=chas,high_confidence=large_range)
    dollar_est=(np.e**log_estimate)*1000*SCALER_FACTOR
    dollar_upper=(np.e**upper)*1000*SCALER_FACTOR
    dollar_lower=(np.e**lower)*1000*SCALER_FACTOR
    rounded_est=round(dollar_est,-3)
    rounded_hi=round(dollar_upper,-3)
    rounded_low=round(dollar_lower,-3)
    print(f"Estimated property value is {rounded_est}")
    print(f"At {conf}% confidence the value range is")
    print(f"USD {rounded_low} at lower end to USD {rounded_hi} at high end")


In [25]:
get_dollar_estimate(rm=3,ptratio=20,chas=True,large_range=True)

Estimated property value is 442000.0
At 95% confidence the value range is
USD 304000.0 at lower end to USD 643000.0 at high end


c:\Users\Shubham\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
